### Problem setting
we consider a three stage news vendor problem
1) first stage:<br>
&emsp; vendors require a fix-investment from agents:<br>
        &emsp;&emsp;variable:<br>
            &emsp;&emsp; &emsp;p\in (0,10) - units of investment<br>
        &emsp;&emsp;coeffients:<br>
            &emsp;&emsp; &emsp; 2 - cost per unit of investment<br>
            
2) second stage<br>
    agents buy news paper from vendors before retailing it, <br>
    the available quantity depends on the fix investmentthe:<br>
        variable:<br>
            x \in (0,10*p)- units of news paper<br>
        coeffients:<br>
            -1 - cost per unit of news paper<br>
        random variable:<br>
3) third stage, agents retail news paper to individuals:<br>
&emsp; &emsp; variables:<br>
&emsp; &emsp; &emsp; z $\in (0,x)$, number of sold news paper<br>
&emsp; &emsp; random variables:<br>
&emsp; &emsp; &emsp; d $\in (0,\inf)$, demand<br> 
      

#### Stage Scenario Data

In [27]:
'''
Template of scenario data structure
Let x - first stage r.v. 
    y - second stage r.v
    z, d - third stage r.v. 

scenario['third_stage'] = {
    'third_stage':{
        'random_variables':[ 'z','d' ], #list of varaible_name
        'conditions': [ (x_1,y_1),... ]
            # list of realized values of previous random variables
            # = second_stage:condition x second_stage:value
        'values':[ (z_1,d_1),... ]
            # list of all posible values to current random variables
            # for the convience, 
            # we don't decomposite each value pair by r.v.
            # and avoid using joint probabiity representation
        'probabilities': {
                i: [(j,probability), ...],
                .
                .
                .
                num_conditions: [ (j, probibility),... ]
                    # i idx to thrid_stage:condition
                    # j idx to third_stage:values
                    # num_conditions for the non-conditional case
                    # num_conditions:[] if not provided
        }
            # dict of the probabilies responding to third_stage:value
    }
}
# the first two stages can ignore keywords accordingly,
# e.g. scenario['first_stage'] is empty coz there is no randomness
'''
scenario = {}

scenario['second_stage'] = {
    'random_variables': ['cap'],
    'conditions': [],
    'values':[2, 4, 6],
    'probabilies':{
        0: [(0,0.5), (1,0.4),(2,0.1)]
    }
}

scenario['third_stage'] = {
    'random_variables': ['d'],
    'conditions': scenario['second_stage']['values'],
    'conditions_prob':[prob for _,prob in scenario['second_stage']['probabilies'][0] ],
    'values': [10, 14, 16, 18, 22],
    'probabilies':{
        0: [(0,0.5), (1,0.4), (3,0.1)],
        1: [(1,0.4), (2,0.3), (3,0.3)],
        2: [(2,0.4), (3,0.4), (4,0.2)]
    }
}

#### Benchmark

In [45]:
import numpy as np
import math

# decision policy -> a reward r.v.
# integer for all variables
q = 4
# 22 is the upper bound for the potential demands
# supply <= demands according to the problem setting
# q*cap <= d_max, where (cap,d_max) = (2,18), (4,18), (6,22)

# x = policy_x(q,cap)
# we use table to represent the policy
x = { q:{cap: min(q*cap,10) for cap in [2,4,6]} for q in range(1,10)}
# policy that agent buy as much as possible if supply is less than 10
#  otherwise buy 10 units


def ssd_benchmark_by_q_x( q, x, scenario):
    benchmark = {}
    benchmark['first_stage'] = {
        'r': [-q*0.3],
        's': [1.0]
    }
    
    # tem_num_condition = len(scenario['second_stage']['conditions'])
    benchmark['second_stage'] = {
        'r': [-x[q][cap] for cap in scenario['second_stage']['values']],
        's': [ prob for _,prob in scenario['second_stage']['probabilies'][0]]
    }
    
    conditions = scenario['third_stage']['conditions']
    values = scenario['third_stage']['values']
    
    benchmark['third_stage'] = {
        'conditions': scenario['third_stage']['conditions'],
        'r': { 
                c_idx:[ 1.5*min(q*conditions[c_idx],\
                                    values[v_idx]) for v_idx,p in prob_v] \
                  for c_idx,prob_v in scenario['third_stage']['probabilies'].items()
        },
        's': { 
                c_idx:[ p for v_idx,p in prob_v] \
                  for c_idx,prob_v in scenario['third_stage']['probabilies'].items()
        }
    }
    
    return benchmark
    
print(ssd_benchmark_by_q_x(q, x, scenario))

{'first_stage': {'r': [-1.2], 's': [1.0]}, 'second_stage': {'r': [-8, -10, -10], 's': [0.5, 0.4, 0.1]}, 'third_stage': {'conditions': [2, 4, 6], 'r': {0: [12.0, 12.0, 12.0], 1: [21.0, 24.0, 24.0], 2: [24.0, 27.0, 33.0]}, 's': {0: [0.5, 0.4, 0.1], 1: [0.4, 0.3, 0.3], 2: [0.4, 0.4, 0.2]}}}


#### Optimal policy of $x$ by solve two-stage subproblems
Given a determined $q$<br>
Distribution of scenarios is given as the above<br>
No SSD contraints to consider

In [44]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def second_stage_policy(q, scenario)->(dict,float):
    policy_x = {}
    r_x = {}
    conditions = scenario['third_stage']['conditions']
    values = scenario['third_stage']['values']
    # first stage
    for c_idx,list_v_p in scenario['third_stage']['probabilies'].items():
        cap = conditions[c_idx]
        upper_bound = q*cap
        # upper_bound for x
        m = gp.Model(f'second_stage_cap_{cap}')
        m.Params.LogToConsole = 0
        x = m.addVar(vtype = GRB.CONTINUOUS, name = 'x')
        
        m.addConstr(x<=upper_bound)
        x_d_aug = {}
        
        for v_idx,prob in list_v_p:
            d = values[v_idx]
            tem_x_d_aug = m.addVar(vtype = GRB.CONTINUOUS, name = f"min(x,{d})")
            # an auxiliary variable for min(x,d)
            
            x_d_aug[d] = tem_x_d_aug
            # store in the dict 
            
            m.addConstr(tem_x_d_aug<=x)
            m.addConstr(tem_x_d_aug<=d)

        m.setObjective(-x+sum( 1.5*x_d_aug[values[v_idx]]*prob for v_idx, prob in list_v_p) , GRB.MAXIMIZE)
        m.optimize()
        policy_x[cap] = x.X
        r_x[cap] =m.getObjective().getValue()
    return policy_x, sum( r_x[conditions[i]]*scenario['third_stage']['conditions_prob'][i] for i in  range(3))
print('q || optilmal policy of q || total reward')
for q in range(1,10):
    policy_x, expected_r = second_stage_policy(q, scenario)
    print(q,'||', policy_x, '||',expected_r-0.3*q)

q || optilmal policy of q || total reward
1 || {2: 2.0, 4: 4.0, 6: 6.0} || 1.3000000000000005
2 || {2: 4.0, 4: 8.0, 6: 12.0} || 2.600000000000001
3 || {2: 6.0, 4: 12.0, 6: 16.0} || 3.800000000000001
4 || {2: 8.0, 4: 14.0, 6: 16.0} || 4.400000000000001
5 || {2: 10.0, 4: 14.0, 6: 16.0} || 4.600000000000001
6 || {2: 10.0, 4: 14.0, 6: 16.0} || 4.300000000000002
7 || {2: 10.0, 4: 14.0, 6: 16.0} || 4.000000000000002
8 || {2: 10.0, 4: 14.0, 6: 16.0} || 3.7000000000000015
9 || {2: 10.0, 4: 14.0, 6: 16.0} || 3.4000000000000017


Obviously, for $q>=5$, the optimal policy is not sensitive to $q$ anymore, that is, the second stage is not sensitive to the upper bound $q*cap$ <br>
Thus, $q=5$ and $x(q,\cdot)$ = { 2->10, 4->14.0, 6->16} is optimal solution when no consideration of SSD 

#### Optimal benchmark

In [48]:
q = 5
x = {}
x[q],_ = second_stage_policy(q, scenario)

optimal_benchmark = ssd_benchmark_by_q_x(5, x, scenario)
print(optimal_benchmark)

{'first_stage': {'r': [-1.5], 's': [1.0]}, 'second_stage': {'r': [-10.0, -14.0, -16.0], 's': [0.5, 0.4, 0.1]}, 'third_stage': {'conditions': [2, 4, 6], 'r': {0: [15.0, 15.0, 15.0], 1: [21.0, 24.0, 27.0], 2: [24.0, 27.0, 33.0]}, 's': {0: [0.5, 0.4, 0.1], 1: [0.4, 0.3, 0.3], 2: [0.4, 0.4, 0.2]}}}


#### Breakpoints of SSD benchmark

In [ ]:
# agent buy x units
coefficient_first_stage = [-1]
# price for agent
bound_first_stage = [0,1000]
# units the agent can buy

# agent sell u units
coefficient_second_stage = [1.5]
bound_second_stage = [0,1000]
# it is subject to undeterministic quantaty
# u<=d
# u<=x

# probability distribution for the demand d
# (d,probability)
scenario_second_stage = [(10,0.1),(14,0.4),(16,0.3),(18,0.2)]

# since the optimal x is on the interval (10,18) due to the simplicity
# alternatively, blender decomposition can solve problem too
# however, fit in all possible x can give a good snapshot of the benchmark

benchmarks = [ [ (0.5*min(x,d)-max(x-d,0),prob) for d,prob in scenario_second_stage ] for x in range(10,19)] 
# for sake convinence:
variables = { 'first_stage':['x'],
             'second_stage':['u']
           }
coefficients = {'x':-1,'u':1.5}
random_variables = ['d']